<a href="https://colab.research.google.com/github/LidiiaMelnyk95/phdproject/blob/main/German_bert_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
!pip install -U -q PyDrive
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/file/d/1QaCUK0AvKlAnxxeuYhCunVHnzO4fmNZl/view?usp=sharing'
id = '1QaCUK0AvKlAnxxeuYhCunVHnzO4fmNZl'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_de.csv')  
df = pd.read_csv('train_de.csv', delimiter = None, index_col = False,header = 0)

In [ ]:
df.dropna(axis=0,subset = ['Statement/No statement'], inplace = True)
df.shape
df.reset_index(drop=True, inplace=True)

In [ ]:
train_df, remaining = train_test_split(df, random_state = 42, train_size = 0.75)
valid_df, _ = train_test_split(remaining, random_state = 42, train_size = 0.25)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


In [ ]:
# create custom dataset class
class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        
        self.labels = labels
        self.text = text

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels [idx]
        data = self.text [idx]
        sample = {"Text": data, "Class": label}
        return sample



In [ ]:
# define data set object
!pip install transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

TD = CustomTextDataset(train_df['Comment'].values,train_df['Statement/No statement'].values)
TD_val = CustomTextDataset(valid_df['Comment'].values, valid_df['Statement/No statement'].values)

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 50.2 MB/s 
     |████████████████████████████████| 596 kB 37.2 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
TD.text

array(['Der Schwachsinn wird erst richtig deutlich, wenn man hört, das Gaststätten überall in Deutschland nun dutzende verschiedene Toiletten einführen sollen, um auch all diese angeblich 60 verschiedene Geschlechter zu berücksichtigen hahahaha..... das müsst ihr euch dann erst mal in der Praxis vorstellen, bis dann die richtige Toilette gefunden hast, hast dir sicher schon in die Hose gemacht hahahahaha.... und wie groß werden zukünftig dann Gaststätten sein, da zahlst ja 80&amp; der Pacht allein schon für die Toiletten hahahaha..... man merkt, Grüne und Linke haben mit Wirtschaft nicht viel am Hut, die sind ja zumeist beim Staat beschäftigt weil die in Firmen keiner haben will, höchstens in Kombinaten und Kolchosen :-)',
       'Die Taten der Diktatoren sind Fakten belegt durch ihr eigenes handeln. Der Vergleich hinkt also. Patti kann nur erzählen aber nicht direkt handeln.<br>Klar wenn es ihm wichtig gewesen wäre hätte er es aus eigener Tasche bezahlen können, damit wären wohl alle 

In [ ]:
train_dataset = TD

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

model = AutoModelForMaskedLM.from_pretrained("bert-base-german-cased")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    weight_decay=0.01)               # strength of weight decay


trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=TD_val, 
    tokenizer = tokenizer          
)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1314
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 495


ValueError: ignored

In [ ]:
trainer.evaluate()
